### 구글맵 평점 및 리뷰 크롤링
- 언어: Python
- 라이브러리: pandas, numpy, tqdm
- 데이터베이스: AWS rds, mysql
- API: google map

#### 1. 사용 라이브러리 로드

In [3]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import urllib
from urllib.request import Request, urlopen
import googlemaps
from dotenv import load_dotenv
import os

load_dotenv()

True

#### 2. 반려동물 동반가능 문화시설 데이터 전처리

In [4]:
data = pd.read_csv("./data/preprocessed_per_culture_site.csv")
data.head()

,Unnamed: 0,시설명,카테고리,주소,위도,경도
0,0,100세약국,동물약국,서울특별시 영등포구 당산동6가 217-4,37.533260,126.903741
1,1,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
2,2,1층메디컬약국,동물약국,서울특별시 금천구 시흥동 889-8,37.453043,126.901712
3,3,1층이화약국,동물약국,서울특별시 성동구 하왕십리동 1070,37.566590,127.024082
4,4,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196


In [5]:
data.drop(["Unnamed: 0"], axis=1, inplace=True)
data.head()

,시설명,카테고리,주소,위도,경도
0,100세약국,동물약국,서울특별시 영등포구 당산동6가 217-4,37.533260,126.903741
1,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
2,1층메디컬약국,동물약국,서울특별시 금천구 시흥동 889-8,37.453043,126.901712
3,1층이화약국,동물약국,서울특별시 성동구 하왕십리동 1070,37.566590,127.024082
4,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196


In [6]:
data = data[data["카테고리"] != "동물약국"]
data.head()

,시설명,카테고리,주소,위도,경도
1,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
4,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196
6,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063
7,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867
8,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256


In [7]:
search_datas = []
for index, each in data.iterrows():
    search_datas.append(str(each["시설명"]) + ", " + str(each["주소"]))

print(len(search_datas), search_datas)

2120 ['100코달리 와인바, 서울특별시 중구 신당동 292-152', '21세기동물병원, 서울특별시 용산구 보광동 259-1', '24시 강서 젠틀리동물의료센터, 서울특별시 강서구 마곡동 747', '24시 굿케어동물의료센터, 서울특별시 관악구 봉천동 1673-21', '24시 글로리 동물병원, 서울특별시 금천구 독산동 145-13', '24시 동물응급의료센터, 서울특별시 중랑구 중화동 207', '24시 명동물메디컬센터, 서울특별시 구로구 개봉동 492', '24시 산들산들 동물병원, 서울특별시 마포구 아현동 424-14', '24시 서울 탑 동물병원, 서울특별시 양천구 신정동 1182-11', '24시 스마트동물병원, 서울특별시 강남구 신사동 587-13', '24시 시유동물메디컬센터, 서울특별시 용산구 이촌동 302-88', '24시 신도림S동물의료센터, 서울특별시 구로구 신도림동 435-10', '24시 안박동물병원, 서울특별시 강서구 등촌동 628-13', '24시 애니동물병원, 서울특별시 성북구 보문동5가 3-1', '24시 지구촌 동물메디컬 센터, 서울특별시 구로구 구로동 505-17', '24시 청담우리 동물병원, 서울특별시 강남구 삼성동 70-6', '24시 SNC 동물메디컬센터, 서울특별시 강남구 역삼동 727-8', '24시루시드동물메디컬센터, 서울특별시 강북구 미아동 42-11', '24시센트럴동물메디컬센터, 서울특별시 성동구 행당동 318-36', '24시스마트동물메디컬센터, 서울특별시 은평구 응암동 91-5', '24시아프리카동물메디컬센터, 서울특별시 강서구 등촌동 664-1', '24시연동물의료센터, 서울특별시 강서구 화곡동 1060-30', '24시예담동물의료센터, 서울특별시 은평구 역촌동 7-15', '24시예스동물병원, 서울특별시 서초구 반포동 20-45', '24시잠실ON동물의료센터, 서울특별시 송파구 잠실동 175-6', '24시지구촌동물메디컬센터, 서울특별시 구로구 구로동 505-17', '24시OK동물병원, 서울특별

#### 3. googlemaps 라이브러리를 이용한 시설명별 리뷰 데이터 가공

In [8]:
def getGoogleMapPlaceId(gmap, search_data):
    geocode = gmap.geocode(address=search_data)
    return geocode[0]['place_id']

In [19]:
def getGoogleMapReviews(gmap, place_id):
    try:
        detail = gmap.place(place_id, language="ko")
    except:
        detail = {"result": []}
    return detail['result']

In [20]:
google_map_key = os.getenv("google_map_key")
gmap = googlemaps.Client(key=google_map_key)

review_list = []

for search_data in tqdm(search_datas):
    place_id = getGoogleMapPlaceId(gmap, search_data)
    reviews = getGoogleMapReviews(gmap, place_id)
    if 'reviews' in reviews:
        for review in reviews['reviews']:
           review_dict = {}
           review_dict["평점"] = review["rating"]
           review_dict["리뷰"] = review["text"]
           review_list.append(review_dict)

print(len(review_list), review_list)

  0%|          | 0/2120 [00:00<?, ?it/s]

2455 [{'평점': 5, '리뷰': '신당동도  이런 분위기좋고\n핫 한곳이 있었다니..ㅋㅋ\n사장님 부부도 너무친절하시고\n음식도 예쁘고 다 ~ 맛있었음'}, {'평점': 4, '리뷰': '분위기 좋고, 와인 가격도 합리적이네요'}, {'평점': 5, '리뷰': ''}, {'평점': 3, '리뷰': ''}, {'평점': 5, '리뷰': ''}, {'평점': 1, '리뷰': ''}, {'평점': 3, '리뷰': '24시간 이라 주말에 방문하기 편리함\n첫 예방접종인데 딱 접종만 하고 끝.\n기본 진찰도 없어 당황.\n로비에 있던 고양이 공격적임'}, {'평점': 4, '리뷰': '24시라 좋아요. 새벽에 강아지가 출혈이 있어서 깜짝 놀랐는데 선생님이 친절하게 설명해주셔서 안심하고 바로 입원시켰어요. 병원비가 비싸다고들 하던데 전 뭐..이정도 서비스에 아깝지 않은 비용이었어요~'}, {'평점': 5, '리뷰': '푸들12년차 작년10월~13년차 지금까지 용혈성빈혈로 수혈2회에 큰일을 치뤘는데..의사쌤께서 넘 꼼꼼하게 잘 치료해주시어 막바지 검진 체크만 하고 있어요\n간호쌤들도 친철하게 잘 봐주시고 믿고 맡길수 있는 곳입니다'}, {'평점': 1, '리뷰': '24시 장점은 있지만 하루 반나절 입원하고 100만원 가까이 비용이은 너무 과도하다. 수술한것도 아니고 새벽에 전화 와17만원 비용이 예상된다고 하더니  수술 한것도 아닌데 100만원 가까이 나왔어요.'}, {'평점': 5, '리뷰': '냐옹이 요로결석이 심해서 급한 마음에 가장 가까운 병원으로 찾아갔었는대 원장쌤이 생각보다 너무 친절하고 실력이 좋으셨습니다. 이번이 네번째 수술이었는대 냥옹님이 집 오자마자의 컨디션이 수술한 아이가 맞나 싶을정도로 너무 편안하고 아무일없었단것 처럼 잘 지내서 대화는 못하지만 실력과 정성을 같이 인정하는것 같아요. 수술에 대해서는 정말 적극 추천합니다.'}, {'평점': 5, '리뷰': '퇴근이 항상 늦어서 야간에 진료를 받게 되었는데 안내해주시는 선생님들, 진료해주시는 

In [41]:
data = pd.DataFrame(review_list)
data.head()

,평점,리뷰
0,5,신당동도 이런 분위기좋고\n핫 한곳이 있었다니..ㅋㅋ\n사장님 부부도 너무친절하시...
1,4,"분위기 좋고, 와인 가격도 합리적이네요"
2,5,
3,3,
4,5,


In [45]:
data.to_csv("./data/reviews.csv")